## Import Library

In [32]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn import metrics

## Defining a Custom Dataset

In [5]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx]
        return {
            "x": torch.tensor(current_sample, dtype=torch.float),
            "y": torch.tensor(current_target, dtype=torch.long)
        }

In [4]:
## Creating dummy data
data, targets = make_classification(n_samples=1000)

In [7]:
## Stratified split
train_data, test_data, train_targets, test_targets = train_test_split(
    data, targets, stratify=targets)

In [8]:
## Creating Dataset
train_dataset = CustomDataset(train_data, train_targets)
test_dataset = CustomDataset(test_data, test_targets)

In [9]:
## Creating Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, num_workers=2)

In [15]:
model = lambda x,w,b: torch.matmul(x,w) + b

In [16]:
train_data.shape

(750, 20)

In [26]:
W = torch.randn(20,1, requires_grad = True)
b = torch.randn(1, requires_grad = True)
learning_rate = 1e-3

for epoch in range(10):
    epoch_loss = 0
    counter = 0
    for data in train_loader:
        xtrain = data["x"]
        ytrain = data["y"]
            
        output = model(x=xtrain, w=W, b=b)
        loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
        epoch_loss += loss.item()
        loss.backward()
        
        with torch.no_grad():
            W = W - learning_rate*W.grad
            b = b - learning_rate*b.grad
            
        W.requires_grad_(True)
        b.requires_grad_(True)
        counter += 1
        
    print(epoch, epoch_loss/counter)

0 15.981553386500542
1 6.919846861286366
2 3.1123846014763448
3 1.4483085493061771
4 0.7075563029267211
5 0.37370014080698505
6 0.2216868143456869
7 0.1518167442335014
8 0.11940561735784595
9 0.1042313200590062


In [34]:
outputs = []
labels = []
with torch.no_grad():
    for data in test_loader:
        xtest = data["x"]
        ytest = data["y"]
        
        output = model(x=xtest, w=W, b=b)
        labels.append(ytest)
        outputs.append(output)

In [36]:
metrics.roc_auc_score(torch.cat(labels).view(-1), torch.cat(outputs).view(-1))

0.9441244239631337